In [6]:
from openai import OpenAI
import os
import dotenv
import json


In [7]:
dotenv.load_dotenv("test.env")

True

# Simple Chatbot Try

In [8]:
# Load your API key from an environment variable
client = OpenAI(api_key=os.getenv('OPENAI_API'))

In [9]:
# Create an instruction for our bot
conversation = [{"role": "system", "content": "You are a chemistry assistant for undergrad students."}]

# For single question answer array
print("I am your chemistry assistant. How can I help you?")
inputquestion = input("Question: ")

conversation.append({"role": "user", "content": inputquestion})

# Getting response from OpenAI for single question answer
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=conversation
)

assistant_response = response.choices[0].message.content
print(f"Assistant: {assistant_response}")

In [10]:
# For multiple questions, it will remember the previous conversation
# Question Panel will pop up, write your question there
while True:
    inputquestion = input("Question: ")
    
    if inputquestion.lower() == 'stop': break
    
    conversation.append({"role": "user","content": inputquestion})
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=conversation
    )
    
    assistant_response = response.choices[0].message.content
        
    conversation.append({"role": "assistant", "content": assistant_response})
        
    print(f"Assistant: {assistant_response}")

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

# Creating JSON file for Q&A pairs

In [ ]:
def read_sentences(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
        sentences = text.split('.')[:25]  # Split by '.' and take the first 25 sentences
    return ' '.join(sentences)

In [ ]:
def read_sentences_in_chunks(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
        sentences = text.split('.')
        for i in range(0, len(sentences), 25):
            yield ' '.join(sentences[i:i+25])

def generate_qa(text):
    response = openai.Completion.create(
        model="text-davinci-003",  # You can specify the model here
        prompt=f"Generate three questions and answers based on the following text in German: {text}",
        max_tokens=500,
        n=1,
        stop=None
    )
    return response.choices[0].text.strip()

In [ ]:
def generate_qa(text):
    conversation = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": f"Generate three questions and answers based on the following text in German: {text}"}
    ]
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=conversation,
        max_tokens=500
        
    )
    last_message = response['choices'][0]['message']['content'] if response.choices else "No response generated."
    return last_message

In [ ]:
def save_to_json(qa_pairs, output_file):
    with open(output_file, 'w', encoding='utf-8') as file:
        json.dump({"questions": qa_pairs}, file, ensure_ascii=False, indent=4)

In [ ]:
def process_file(file_path, output_file):
    for text_chunk in read_sentences_in_chunks(file_path):
        qa_text = generate_qa(text_chunk)
        qa_pairs = [{"question": q.strip(), "answer": a.strip()} for q, a in zip(qa_text.split('\n')[::2], qa_text.split('\n')[1::2])]
        save_to_json(qa_pairs, output_file)



In [ ]:
# Specify the file paths and start processing
file_path = 'chapter_1.txt_extracted_text2.txt_remove_unwanted_sentences_cleaned.txt'  # Adjust this to your file's name
output_file = 'QA_dataset.json'
process_file(file_path, output_file)

TypeError: 'ChatCompletion' object is not subscriptable

### TODO: Translation to english

In [ ]:
a

In [ ]:
from deep_translator import GoogleTranslator

In [ ]:
import pdfplumber
import json

In [ ]:
pdf = pdfplumber.open("Abs_1.pdf")

In [ ]:
page11 = pdf.pages[11]
page11.extract_text()

'1. Das ideale Gas\nD\nas ideale Gas stellt das einfachste physikalische System dar, anhand des- IN DIESEM ABSCHNITT\nsen sich die Grundlagen und Aussagen der Thermodynamik anschaulich dis-\n• Was sind Zustandsgrößen und wozu\nkutieren lassen. Als erstes werden daher wir die sog. Zustandsgleichung des\ndienen sie?\nidealen Gases kennenlernen. Dieses Gasgesetz eignet sich ideal (sic), um\nwichtige Zusammenhänge zwischen elementaren Zustandsgrößen der Ther-\n• Was besagt die Zustandsgleichung?\nmodynamik herzustellen und zu verstehen. Am Ende dieses Kapitels diskutie-\nren wir die Grundlagen der kinetischen Gastheorie. Sie erlaubt die Herleitung • Der Nullte Hauptsatz der\ndes idealen Gasgesetzes aus sehr einfachen mikroskopischen Modellvorstel- Thermodynamik\nlungen.\n• Was ist ein ideales Gas?\n• Die kinetische Gastheorie\nKAPITEL 1 15'

In [ ]:
def extract(page):
    """Extract PDF text and Delete in-paragraph line breaks."""
    # Get text
    extracted = page.extract_text()
    # Delete in-paragraph line breaks
    extracted = extracted.replace(".\n", "**/m" # keep par breaks
                        ).replace(". \n", "**/m" # keep par breaks
                        ).replace("\n", "" # delete in-par breaks     
                        ).replace("**/m", ".\n\n") # restore par break
    return extracted


In [ ]:
print(extract(page11))

1. Das ideale GasDas ideale Gas stellt das einfachste physikalische System dar, anhand des- IN DIESEM ABSCHNITTsen sich die Grundlagen und Aussagen der Thermodynamik anschaulich dis-• Was sind Zustandsgrößen und wozukutieren lassen. Als erstes werden daher wir die sog. Zustandsgleichung desdienen sie?idealen Gases kennenlernen. Dieses Gasgesetz eignet sich ideal (sic), umwichtige Zusammenhänge zwischen elementaren Zustandsgrößen der Ther-• Was besagt die Zustandsgleichung?modynamik herzustellen und zu verstehen. Am Ende dieses Kapitels diskutie-ren wir die Grundlagen der kinetischen Gastheorie. Sie erlaubt die Herleitung • Der Nullte Hauptsatz derdes idealen Gasgesetzes aus sehr einfachen mikroskopischen Modellvorstel- Thermodynamiklungen.

• Was ist ein ideales Gas?• Die kinetische GastheorieKAPITEL 1 15


In [ ]:
translate = GoogleTranslator(source="auto", target="en").translate

In [ ]:
from nltk.tokenize import sent_tokenize

In [ ]:
def translate_extracted(Extracted):
    """Wrapper for Google Translate with upload workaround."""
    # Set-up and wrap translation client
    translate = GoogleTranslator(source='auto', target='en').translate
    # Split input text into a list of sentences
    sentences = sent_tokenize(Extracted)
    # Initialize containers
    translated_text = ''
    source_text_chunk = ''
    # collect chuncks of sentences, translate individually
    for sentence in sentences:
    # if chunck + current sentence < limit, add the sentence
        if ((len(sentence.encode('utf-8')) +  len(source_text_chunk.encode('utf-8')) < 5000)):
            source_text_chunk += ' ' + sentence
        # else translate chunck and start new one with current sentence
        else:
            translated_text += ' ' + translate(source_text_chunk)
            # if current sentence smaller than 5000 chars, start new chunck
            if (len(sentence.encode('utf-8')) < 5000):
                source_text_chunk = sentence
            # else, replace sentence with notification message
            else:
                message = "<<Omitted Word longer than 5000bytes>>"
                translated_text += ' ' + translate(message)
                # Re-set text container to empty
                source_text_chunk = ''
  # Translate the final chunk of input text, if there is any valid   text left to translate
    if translate(source_text_chunk) != None:
        translated_text += ' ' + translate(source_text_chunk)
    return translated_text

In [ ]:
extracted = extract(pdf.pages[12])
translated = translate_extracted(extracted)[:500]
print(translated)

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/PY3/english.pickle[0m

  Searched in:
    - '/Users/serlink/nltk_data'
    - '/opt/homebrew/Caskroom/mambaforge/base/envs/hertel_pc/nltk_data'
    - '/opt/homebrew/Caskroom/mambaforge/base/envs/hertel_pc/share/nltk_data'
    - '/opt/homebrew/Caskroom/mambaforge/base/envs/hertel_pc/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************
